In [1]:
%run react_functions.ipynb

In [2]:
import shutil
import os

if shutil.which("g16"):
    print("✅ 环境检查通过：系统已找到 Gaussian (g16)。")
else:
    print("❌ 警告：未找到 g16，请检查环境变量配置。")

✅ 环境检查通过：系统已找到 Gaussian (g16)。


In [4]:
# ==========================================
# 氨气 (NH3) + 碘甲烷 (MeI) 快速验证方案
# ==========================================
import numpy as np
from rdkit import Chem, Geometry
from rdkit.Chem import AllChem

print(">>> 1. 构建 NH3 + MeI 体系...")

# --- 1. 手动构建分子 (不依赖随机生成) ---
mol_nh3 = Chem.MolFromSmiles("N") 
mol_nh3 = Chem.AddHs(mol_nh3)
# NH3 也是 4 个原子，先 Embed
AllChem.EmbedMolecule(mol_nh3, randomSeed=42)

mol_mei = Chem.MolFromSmiles("CI") 
mol_mei = Chem.AddHs(mol_mei)
AllChem.EmbedMolecule(mol_mei, randomSeed=42)

# --- 2. 几何对齐 (让孤对电子对准 C) ---
conf_nh3 = mol_nh3.GetConformer()
conf_mei = mol_mei.GetConformer()

# A. 摆放 NH3 (N 在原点，H 在 Z 轴负方向，像个底座)
# N 原子
conf_nh3.SetAtomPosition(0, Geometry.Point3D(0, 0, 0))
# 3 个 H 原子 (四面体角度，键长 ~1.0)
# 简单的三角锥几何
h_offsets = [
    (0.0, 0.94, -0.38),
    (0.81, -0.47, -0.38),
    (-0.81, -0.47, -0.38)
]
for i in range(1, 4): # H 的索引是 1,2,3
    x, y, z = h_offsets[i-1]
    conf_nh3.SetAtomPosition(i, Geometry.Point3D(x, y, z))

# B. 摆放 MeI (C 在 Z=3.0, I 在 Z=5.2)
c_idx = [a.GetIdx() for a in mol_mei.GetAtoms() if a.GetSymbol() == 'C'][0]
i_idx = [a.GetIdx() for a in mol_mei.GetAtoms() if a.GetSymbol() == 'I'][0]
h_indices = [a.GetIdx() for a in mol_mei.GetAtoms() if a.GetSymbol() == 'H']

# C 稍微偏一点点 (0.1) 避开 180 度死角
conf_mei.SetAtomPosition(c_idx, Geometry.Point3D(0.1, 0, 3.0)) 
conf_mei.SetAtomPosition(i_idx, Geometry.Point3D(0.2, 0, 5.2))

# C 上的 H (伞状，指向 N)
h_offsets_c = [(0.1, 1.03, 2.7), (0.99, -0.51, 2.7), (-0.79, -0.51, 2.7)]
for idx, off in zip(h_indices, h_offsets_c):
    conf_mei.SetAtomPosition(idx, Geometry.Point3D(off[0], off[1], off[2]))

# 合并
nh3_sys = Chem.CombineMols(mol_nh3, mol_mei)

# 自动获取序号
n_idx = [a.GetIdx()+1 for a in nh3_sys.GetAtoms() if a.GetSymbol() == 'N'][0]
c_idx = [a.GetIdx()+1 for a in nh3_sys.GetAtoms() if a.GetSymbol() == 'C'][0]
i_idx = [a.GetIdx()+1 for a in nh3_sys.GetAtoms() if a.GetSymbol() == 'I'][0]

print(f">>> 对齐完成！N={n_idx}, C={c_idx}, I={i_idx}")
# view_with_labels(nh3_sys, "NH3 + MeI Aligned") # 想看图可以取消注释

# --- 3. 快速预览扫描 (Coarse Scan) ---
print(f"\n>>> 2. 启动快速预览扫描 (Preview Scan)")
print(f"    策略：步长加大，步数减少，只求看个趋势")

# 扫描 15 步，每步缩短 0.1 埃 (总共缩短 1.5 埃，足够覆盖过渡态)
scan_lines = [
    f"{n_idx} {c_idx} S 15 -0.10", 
    # f"{n_idx} {c_idx} {i_idx} F",   # 依然加上角度冻结保险
]

gjf_preview = create_advanced_scan_gaussian_input(
    nh3_sys,
    "NH3_Preview",
    scan_lines=scan_lines,
    charge=0, # 整体电荷 0
    method="#PM7 opt(modredundant)", # 极速模式
    filename="nh3_preview.gjf"
)

# 运行 (这个应该非常快，1分钟左右)
run_gaussian_job(gjf_preview)

# --- 4. 检查结果 ---
print("\n>>> 3. 检查能量曲线...")
points = read_scan_output("nh3_preview.log")

if points:
    energies = [p['energy'] for p in points]
    max_e = max(energies)
    max_idx = energies.index(max_e)
    
    # 打印简易曲线
    print(f"最高能量点在第 {points[max_idx]['step']} 步")
    
    # 简单的文本绘图
    min_e = min(energies)
    print("能量趋势图 (文本版):")
    for i, e in enumerate(energies):
        # 归一化到 0-10 颗星
        stars = int((e - min_e) / (max_e - min_e) * 20)
        marker = "<-- TS?" if i == max_idx else ""
        print(f"Step {i+1:2d}: {'*' * stars} {marker}")
        
    print(f"\n>>> 建议：")
    if 0 < max_idx < len(points)-1:
        print("✅ 成功！能量先升后降，这是一个完美的过渡态曲线！")
        print("您可以直接提取第 {} 步的结构去做 TS 优化了。".format(points[max_idx]['step']))
    else:
        print("⚠️ 警告：能量没有出现峰值（一直上升或一直下降）。")
        print("可能需要调整距离范围。")

>>> 1. 构建 NH3 + MeI 体系...
>>> 对齐完成！N=1, C=5, I=6

>>> 2. 启动快速预览扫描 (Preview Scan)
    策略：步长加大，步数减少，只求看个趋势
扫描行: 1 5 S 15 -0.10
柔性扫描Gaussian输入文件已保存为: nh3_preview.gjf
执行命令: g16.exe  nh3_preview.gjf nh3_preview.log
计算完成!

>>> 3. 检查能量曲线...
最高能量点在第 13 步
能量趋势图 (文本版):
Step  1:  
Step  2:  
Step  3:  
Step  4:  
Step  5: * 
Step  6: ** 
Step  7: *** 
Step  8: ***** 
Step  9: ******** 
Step 10: *********** 
Step 11: **************** 
Step 12: ******************* 
Step 13: ******************** <-- TS?
Step 14: ****************** 
Step 15: *************** 
Step 16: *************** 

>>> 建议：
✅ 成功！能量先升后降，这是一个完美的过渡态曲线！
您可以直接提取第 13 步的结构去做 TS 优化了。


In [6]:
import py3Dmol
from rdkit import Chem

def view_with_labels(mol, name):
    """专门用于显示原子序号的可视化函数"""
    mb = Chem.MolToMolBlock(mol)
    view = py3Dmol.view(width=600, height=600)
    view.addModel(mb, 'mol')
    view.setStyle({'stick': {'radius': 0.15}, 'sphere': {'scale': 0.25}})
    view.setBackgroundColor('0xeeeeee')
    
    # 添加原子序号标签
    for i, atom in enumerate(mol.GetAtoms()):
        pos = mol.GetConformer().GetAtomPosition(i)
        view.addLabel(str(i+1), {
            'position': {'x': pos.x, 'y': pos.y, 'z': pos.z},
            'backgroundColor': 'white',
            'fontColor': 'black',
            'fontSize': 14,
            'backgroundOpacity': 0.8
        })
    
    view.zoomTo()
    view.show()
    print(f"=== {name} (带原子序号) ===")


In [7]:
# ==========================================
# NH3 + MeI: TS 精确优化与验证
# ==========================================

# 1. 确保我们有预览扫描的结果
if 'points' in locals() and points:
    # 找到最高点
    max_e = max([p['energy'] for p in points])
    max_point = [p for p in points if p['energy'] == max_e][0]
    
    print(f">>> 提取 Preview Scan 中第 {max_point['step']} 步的结构作为初猜...")
    ts_guess_nh3, _ = extract_scan_point_to_mol(points, max_point['step'], nh3_sys, show_3d=False)
    
    # 2. 生成 TS 优化文件
    # 必须加 calcall (算力常数) 和 noeig (不打印海森矩阵特征向量)
    gjf_ts_nh3 = create_gaussian_input_advanced(
        ts_guess_nh3,
        "NH3_TS_Final",
        charge=0,
        method="#PM7 opt(ts,noeig,calcall)", 
        filename="nh3_ts.gjf"
    )
    
    # 3. 运行
    print(">>> 正在运行 TS 优化 (Berny 算法)...")
    run_gaussian_job(gjf_ts_nh3)
    
    # 4. 读取结果并验证虚频
    results_ts_nh3 = read_gaussian16_output_opt("nh3_ts.log")
    ts_final_nh3 = update_mol_coordinates(ts_guess_nh3, results_ts_nh3)
    
    freqs = results_ts_nh3.get('frequencies')
    print(f"\n>>> 频率检查: {freqs}")
    
    if freqs and freqs[0] < 0:
        print(f"✅ 漂亮！找到唯一的虚频: {freqs[0]:.2f} cm^-1")
        print("这证实了这是一个真正的过渡态！")
        view_with_labels(ts_final_nh3, "NH3 + MeI Transition State")
    else:
        print("❌ 失败：没有找到虚频。")
else:
    print("请先运行 Preview Scan 代码块。")

>>> 提取 Preview Scan 中第 13 步的结构作为初猜...
提取扫描点 13:
能量: 0.04270345 Hartree
Gaussian输入文件已保存为: nh3_ts.gjf
>>> 正在运行 TS 优化 (Berny 算法)...
执行命令: g16.exe  nh3_ts.gjf nh3_ts.log
计算完成!
找到 4 个几何结构部分
成功提取 9 个原子的坐标

>>> 频率检查: [-562.6162, 141.9409, 198.0152]
✅ 漂亮！找到唯一的虚频: -562.62 cm^-1
这证实了这是一个真正的过渡态！


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

=== NH3 + MeI Transition State (带原子序号) ===


In [8]:
# ==========================================
# 阶段四：IRC 反应路径验证
# ==========================================

# 1. 确保我们有优化好的过渡态结构
if 'ts_final_nh3' in locals() and ts_final_nh3:
    print(">>> 启动 IRC (内禀反应坐标) 计算...")
    print("    目的：验证过渡态连接了正确的反应物和产物。")
    
    # 2. 生成 IRC 输入文件
    # maxpoints=15: 向每个方向走15步 (通常足够看清趋势了)
    # stepsize=10: 每步走多远 (单位是 0.1 amu^(1/2)*Bohr)
    # calcall: 为了保证 IRC 走得准，每步算 Hessian (PM7 算得起)
    gjf_irc = create_gaussian_input_advanced(
        ts_final_nh3, # 使用刚刚优化好的 TS 结构作为起点
        "NH3_IRC",
        charge=0,
        method="#PM7 IRC(calcall, maxpoints=15, stepsize=10)", 
        filename="nh3_irc.gjf"
    )
    
    # 3. 运行 (可能需要 5-10 分钟，取决于步数)
    run_gaussian_job(gjf_irc)
    
    # 4. 解析结果
    print("\n>>> IRC 计算结束，正在解析路径...")
    irc_points = read_irc_output("nh3_irc.log")
    
    if irc_points:
        print_irc_results(irc_points)
        
        # 5. 可视化关键帧
        # 提取路径两端的终点 (Endpoint)
        # 路径1的最后一点 (通常是反应物方向)
        print("\n>>> 反应物侧终点 (Path 1 Last Point):")
        path1_end = [p for p in irc_points if p['path_number']==1][-1]
        mol_p1, _ = extract_irc_point_to_mol(irc_points, 1, path1_end['point_number'], show_3d=True)
        
        # 路径2的最后一点 (通常是产物方向)
        print("\n>>> 产物侧终点 (Path 2 Last Point):")
        path2_end = [p for p in irc_points if p['path_number']==2][-1]
        mol_p2, _ = extract_irc_point_to_mol(irc_points, 2, path2_end['point_number'], show_3d=True)
        
        print("\n>>> 结论验证：")
        print("请观察上面两个图：")
        print("图1：是否回到了 NH3 和 MeI 分开的状态？(反应物)")
        print("图2：是否变成了 Me-NH3+ 和 I- 在一起的状态？(产物)")
    else:
        print("❌ 读取 IRC 结果失败。")
else:
    print("请先成功完成 TS 优化步骤。")

>>> 启动 IRC (内禀反应坐标) 计算...
    目的：验证过渡态连接了正确的反应物和产物。
Gaussian输入文件已保存为: nh3_irc.gjf
执行命令: g16.exe  nh3_irc.gjf nh3_irc.log
计算完成!

>>> IRC 计算结束，正在解析路径...
Point Number:   0          Path Number:   1
找到IRC点: 路径 None, 点 None, 能量 = 0.04300106
Point Number:   1          Path Number:   1
找到IRC点: 路径 1, 点 0, 能量 = 0.04238205
Point Number:   2          Path Number:   1
找到IRC点: 路径 1, 点 1, 能量 = 0.04051548
Point Number:   3          Path Number:   1
找到IRC点: 路径 1, 点 2, 能量 = 0.03747879
Point Number:   4          Path Number:   1
找到IRC点: 路径 1, 点 3, 能量 = 0.03344822
Point Number:   5          Path Number:   1
找到IRC点: 路径 1, 点 4, 能量 = 0.02867457
Point Number:   6          Path Number:   1
找到IRC点: 路径 1, 点 5, 能量 = 0.02345398
Point Number:   7          Path Number:   1
找到IRC点: 路径 1, 点 6, 能量 = 0.01809930
Point Number:   8          Path Number:   1
找到IRC点: 路径 1, 点 7, 能量 = 0.01291915
Point Number:   9          Path Number:   1
找到IRC点: 路径 1, 点 8, 能量 = 0.00820739
Point Number:  10          Path Number:   1
找到IRC点: 路

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

扫描点 (1, 15) 3D结构:
IRC路径 1 点 15 已提取，能量: 0.04242126 Hartree

>>> 产物侧终点 (Path 2 Last Point):


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

扫描点 (2, 13) 3D结构:
IRC路径 2 点 13 已提取，能量: 0.03086645 Hartree

>>> 结论验证：
请观察上面两个图：
图1：是否回到了 NH3 和 MeI 分开的状态？(反应物)
图2：是否变成了 Me-NH3+ 和 I- 在一起的状态？(产物)


In [12]:
# ==========================================
# 强制可视化 IRC 终点 (反应物 vs 产物)
# ==========================================

# 1. 重新读取 IRC 数据 (确保数据在内存中)
irc_points = read_irc_output("nh3_irc.log")

if irc_points:
    # ------------------------------------------------------
    # 提取 路径 1 的最后一个点 (推测是产物)
    # ------------------------------------------------------
    # 筛选出 Path 1 的所有点
    path1_points = [p for p in irc_points if p['path_number'] == 1]
    if path1_points:
        # 找最后一个点
        last_p1 = path1_points[-1]
        print(f"\n>>> 路径 1 终点 (Step {last_p1['point_number']})")
        print(f"    能量: {last_p1['energy']:.5f} Hartree (相对TS: {(last_p1['energy']-0.04300)*627.5:.1f} kcal/mol)")
        
        # 转换为分子对象
        mol_p1, _ = extract_irc_point_to_mol(irc_points, 1, last_p1['point_number'], show_3d=False)
        
        # 强制显示
        print("    [结构特征预测]：应该是 C-N 键形成，I 离子断开但吸附在旁边")
        view_with_labels(mol_p1, "IRC Path 1 End (Product?)")
    else:
        print("未找到路径 1 的数据")

    # ------------------------------------------------------
    # 提取 路径 2 的最后一个点 (推测是反应物)
    # ------------------------------------------------------
    # 筛选出 Path 2 的所有点
    path2_points = [p for p in irc_points if p['path_number'] == 2]
    if path2_points:
        # 找最后一个点
        last_p2 = path2_points[-1]
        print(f"\n>>> 路径 2 终点 (Step {last_p2['point_number']})")
        print(f"    能量: {last_p2['energy']:.5f} Hartree (相对TS: {(last_p2['energy']-0.04300)*627.5:.1f} kcal/mol)")
        
        # 转换为分子对象
        mol_p2, _ = extract_irc_point_to_mol(irc_points, 2, last_p2['point_number'], show_3d=False)
        
        # 强制显示
        print("    [结构特征预测]：应该是 NH3 和 MeI 稍微分开，C-I 键还是连着的")
        view_with_labels(mol_p2, "IRC Path 2 End (Reactant?)")
    else:
        print("未找到路径 2 的数据")

else:
    print("错误：无法读取 nh3_irc.log，请检查文件是否存在。")

Point Number:   0          Path Number:   1
找到IRC点: 路径 None, 点 None, 能量 = 0.04300106
Point Number:   1          Path Number:   1
找到IRC点: 路径 1, 点 0, 能量 = 0.04238205
Point Number:   2          Path Number:   1
找到IRC点: 路径 1, 点 1, 能量 = 0.04051548
Point Number:   3          Path Number:   1
找到IRC点: 路径 1, 点 2, 能量 = 0.03747879
Point Number:   4          Path Number:   1
找到IRC点: 路径 1, 点 3, 能量 = 0.03344822
Point Number:   5          Path Number:   1
找到IRC点: 路径 1, 点 4, 能量 = 0.02867457
Point Number:   6          Path Number:   1
找到IRC点: 路径 1, 点 5, 能量 = 0.02345398
Point Number:   7          Path Number:   1
找到IRC点: 路径 1, 点 6, 能量 = 0.01809930
Point Number:   8          Path Number:   1
找到IRC点: 路径 1, 点 7, 能量 = 0.01291915
Point Number:   9          Path Number:   1
找到IRC点: 路径 1, 点 8, 能量 = 0.00820739
Point Number:  10          Path Number:   1
找到IRC点: 路径 1, 点 9, 能量 = 0.00423888
Point Number:  11          Path Number:   1
找到IRC点: 路径 1, 点 10, 能量 = 0.00125684
Point Number:  12          Path Number:   1
找

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

=== IRC Path 1 End (Product?) (带原子序号) ===

>>> 路径 2 终点 (Step 13)
    能量: 0.03087 Hartree (相对TS: -7.6 kcal/mol)
IRC路径 2 点 13 已提取，能量: 0.03086645 Hartree
    [结构特征预测]：应该是 NH3 和 MeI 稍微分开，C-I 键还是连着的


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

=== IRC Path 2 End (Reactant?) (带原子序号) ===


In [13]:
# ==========================================
# 备用：保存结构到本地文件 (.xyz)
# ==========================================
def save_xyz(mol, filename):
    if not mol: return
    num_atoms = mol.GetNumAtoms()
    conf = mol.GetConformer()
    with open(filename, 'w') as f:
        f.write(f"{num_atoms}\n")
        f.write(f"Generated from IRC Log\n")
        for i in range(num_atoms):
            pos = conf.GetAtomPosition(i)
            sym = mol.GetAtomWithIdx(i).GetSymbol()
            f.write(f"{sym} {pos.x:.6f} {pos.y:.6f} {pos.z:.6f}\n")
    print(f"已保存文件: {filename}")

# 保存两个终点
if 'mol_p1' in locals() and mol_p1:
    save_xyz(mol_p1, "path1_end_product.xyz")

if 'mol_p2' in locals() and mol_p2:
    save_xyz(mol_p2, "path2_end_reactant.xyz")

print("\n>>> 你现在可以在左侧文件列表中找到 .xyz 文件，下载后用记事本打开查看坐标，或者用化学软件打开。")

已保存文件: path1_end_product.xyz
已保存文件: path2_end_reactant.xyz

>>> 你现在可以在左侧文件列表中找到 .xyz 文件，下载后用记事本打开查看坐标，或者用化学软件打开。


In [14]:
import numpy as np

def calc_dist(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)

# 手动输入你 XYZ 文件里的坐标 (从你上传的文件中提取)
# 格式: [x, y, z]

# Path 1 (你以为是产物，其实是反应物)
n_p1 = [-0.154, 0.738, 3.696]
c_p1 = [-0.081, 0.391, 1.962]
i_p1 = [0.030, -0.146, -0.732]

# Path 2 (你以为是反应物，其实是产物)
n_p2 = [-0.161, 0.772, 3.866]
c_p2 = [-0.099, 0.475, 2.380]
i_p2 = [0.033, -0.159, -0.799]

d_cn_1 = calc_dist(n_p1, c_p1)
d_ci_1 = calc_dist(c_p1, i_p1)

d_cn_2 = calc_dist(n_p2, c_p2)
d_ci_2 = calc_dist(c_p2, i_p2)

print("=== 谜底揭晓 ===")
print(f"Path 1 (能量 -0.002): C-N = {d_cn_1:.2f} Å,  C-I = {d_ci_1:.2f} Å")
print(f"Path 2 (能量 +0.030): C-N = {d_cn_2:.2f} Å,  C-I = {d_ci_2:.2f} Å")

print("\n=== 结论判定 ===")
if d_cn_1 > d_cn_2:
    print("Path 1 的 C-N 键更长 -> 它是反应物 (Reactant Complex)")
    print("Path 2 的 C-N 键更短 -> 它是产物 (Product Ion Pair)")

=== 谜底揭晓 ===
Path 1 (能量 -0.002): C-N = 1.77 Å,  C-I = 2.75 Å
Path 2 (能量 +0.030): C-N = 1.52 Å,  C-I = 3.24 Å

=== 结论判定 ===
Path 1 的 C-N 键更长 -> 它是反应物 (Reactant Complex)
Path 2 的 C-N 键更短 -> 它是产物 (Product Ion Pair)
